# Trabalho Final - Redes Complexas¶

Código contendo o trabalho final desenvolvido na disciplina de Redes Complexas


https://mplsoccer.readthedocs.io/en/latest/index.html

https://statsbomb.com/

https://fcpython.com/

Adicionar um módulo com Streamlit: https://fcpython.com/data-analysis/building-interactive-analysis-tools-with-python-streamlit

Agrupando jogadores com K-Means Cluster: https://fcpython.com/machine-learning/introduction-to-k-means-with-python-clustering-shot-creators-in-the-premier-league

https://statsbomb.com/what-we-do/hub/free-data/

https://mplsoccer.readthedocs.io/en/latest/gallery/pitch_plots/plot_pass_network.html#sphx-glr-gallery-pitch-plots-plot-pass-network-py

https://mplsoccer.readthedocs.io/en/latest/gallery/pitch_plots/plot_pass_network.html

Sbopen mplsoccer uso: https://mplsoccer.readthedocs.io/en/latest/mplsoccer.statsbomb.html#mplsoccer.statsbomb.Sbopen

In [ ]:
# Instalando o mplsoccer na máquina para utilizar dados do statsbomb e fazer plots no campo de futebol
pip install mplsoccer

A bibliotece mplsoccer fornece recursos para plotar gráficos de futebol em Matplotlib, assim como para importar dados abertos do StatsBomb

In [4]:
# Importando as bibliotecas necessárias
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen
import pandas as pd

In [58]:
# Instanciando um objeto parser para importar dados abertos da StatsBomb
parser = Sbopen()

# Extraindo os dados de competições disponíveis no StatsBomb em um DataFrame
df_competition = parser.competition()

# Exibindo as informações das competições disponíveis
#df_competition.info()

In [59]:
#df_competition

In [60]:
# Verificando quais dessas competições são de futebol feminino
df_female_competitions = df_competition.loc[df_competition['competition_gender'] == "female"]

# Exibindo as competições filtradas
#df_female_competitions

In [61]:
## 
# Extraindo os dados de todas as partidas da competição Women's World Cup - 2023
# competition_id = 72 e season_id = 107
## 
df_matches = parser.match(competition_id = 72, season_id = 107)

# Resumo da quantidade de dados
# df_matches.info()

In [62]:
# df_matches

In [36]:
# Percorrendo o data frame de partidas
#for index, row in df_matches.iterrows():
#    print(row['match_id'], row['match_date'])
    
    # Extrai o Id daquela partida
#    match_id = row['match_id']

    # Extrai os eventos daquela partida fornecendo seu Id para o parser
#   df_event, df_related, df_freeze, df_tatics = parser(match_id)

In [56]:
# Extraindo o data frame contendo os dados da partida específica
df_current_match = df_matches.loc[df_matches['match_id'] == 3904629]

# Extraindo o nome e id do time da casa
match_home_team = df_current_match.iloc[0]["home_team_name"]
match_home_team_id = df_current_match.iloc[0]["home_team_id"]

print(str(match_home_team))
print(str(match_home_team_id))

# Extraindo o nome e id do time de fora
match_away_team = df_current_match.iloc[0]["away_team_name"]
match_away_team_id = df_current_match.iloc[0]["away_team_id"]

print(str(match_away_team))
print(str(match_away_team_id))

Australia Women's
1205
England Women's
865


In [33]:
# Extrai os eventos de uma partida fornecendo seu Id para o parser
df_events, related, freeze, tatics = parser.event(3904629)

In [63]:
# Percorrendo o data frame de eventos da partida
#for index, row in df_events.iterrows():
    #print(row['type_name'])

### Extraindo os dados dos passes

In [96]:
# Procurando por eventos de substituição no jogo - (Time da Casa)
substitutions = df_events.loc[df_events["type_name"] == "Substitution"]
home_team_subs = substitutions.loc[substitutions["team_name"] == match_home_team]

# Extrai o índice do primeiro evento de substituição - (Time da Casa)
ht_first_sub_index = home_team_subs.iloc[0]["index"]

# Extraindo todos os eventos de passe da partida
match_passes = df_events.loc[df_events["type_name"] == "Pass"]
# Extraindo os eventos de passe - (Time da Casa)
home_team_passes = match_passes.loc[match_passes["team_name"] == match_home_team]

# Removendo eventos de Lateral
home_team_passes = home_team_passes.loc[match_passes.sub_type_name != "Throw-in"]

# Os passes bem sucedidos são aqueles nos quais o outcome_name é nulo
# Extraindo os passes bem sucedidos - (Time da Casa)
ht_succesful_passes = home_team_passes.loc[home_team_passes["outcome_name"].isnull()]

## 
# Os passes que aconteceram antes da primeira substituição têm id menor que o do evento de substituição. 
# Os ids dos eventos são sequenciais
## 
ht_first_sub_passes = ht_succesful_passes.loc[ht_succesful_passes["index"] < ht_first_sub_index]

## Extraindo os dados necessários para a rede de passes
# x: coordenada x de início do passe
# y: coordenada y de início do passe
# end_x: coordenada x de fim do passe
# end_y: coordenada y de fim do passe
# player_name: jogador que inicia o passe
# pass_recipient_name: jogador que recebe o passe
##
ht_first_sub_passes = ht_first_sub_passes[['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]

# Formatando o nome das jogadoras para que apenas o sobrenome delas seja exibido
ht_first_sub_passes["player_name"] = ht_first_sub_passes["player_name"].apply(lambda x: str(x).split()[-1])
ht_first_sub_passes["pass_recipient_name"] = ht_first_sub_passes["pass_recipient_name"].apply(lambda x: str(x).split()[-1])

### Calculando o tamanho e localização dos nós

In [ ]:
# Procurando por eventos de substituição no jogo - (Time da Casa)
substitutions = df_events.loc[df_events["type_name"] == "Substitution"]
home_team_subs = substitutions.loc[substitutions["team_name"] == match_home_team]
away_team_subs = substitutions.loc[substitutions["team_name"] == match_away_team]

# Extrai o índice do primeiro evento de substituição - (Time da Casa)
ht_first_sub_index = home_team_subs.iloc[0]["index"]
at_first_sub_index = away_team_subs.iloc[0]["index"]

# Extraindo todos os eventos de passe da partida
match_passes = df_events.loc[df_events["type_name"] == "Pass"]
# Extraindo os eventos de passe - (Time da Casa)
home_team_passes = match_passes.loc[match_passes["team_name"] == match_home_team]
# Extraindo os eventos de passe - (Time de Fora)
away_team_passes = match_passes.loc[match_passes["team_name"] == match_away_team]

# Removendo eventos de Lateral
# Extraindo os eventos de passe - (Time da Casa)
home_team_passes = home_team_passes.loc[match_passes.sub_type_name != "Throw-in"]
# Extraindo os eventos de passe - (Time de Fora)
away_team_passes = away_team_passes.loc[match_passes.sub_type_name != "Throw-in"]

# Os passes bem sucedidos são aqueles nos quais o outcome_name é nulo
# Extraindo os passes bem sucedidos - (Time da Casa)
ht_succesful_passes = home_team_passes.loc[home_team_passes["outcome_name"].isnull()]
# Extraindo os passes bem sucedidos - (Time de Fora)
at_succesful_passes = away_team_passes.loc[away_team_passes["outcome_name"].isnull()]

## 
# Os passes que aconteceram antes da primeira substituição têm id menor que o do evento de substituição. 
# Os ids dos eventos são sequenciais
## 
ht_first_sub_passes = ht_succesful_passes.loc[ht_succesful_passes["index"] < ht_first_sub_index]
at_first_sub_passes = at_succesful_passes.loc[at_succesful_passes["index"] < at_first_sub_index]



In [87]:
# Validando quais são os possíveis resultados para uma tentativa de passe
passes_outcomes = set(away_team_passes["outcome_name"])
passes_outcomes

{'Incomplete', 'Out', 'Unknown', nan}

Um passe pode ser incompleto, para fora, desconhecido ou nan

{'Incomplete', 'Out', 'Unknown', nan}

In [75]:
# Validando quais são os possíveis tipos de passes
passes_subtypes = set(away_team_passes["sub_type_name"])
passes_subtypes

{'Corner',
 'Free Kick',
 'Goal Kick',
 'Interception',
 'Kick Off',
 'Recovery',
 'Throw-in',
 nan}